In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import norm
import matplotlib as matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
## Sætter grænseværdien for animationsstørrelsen op##
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Roterende pendul

Et system med:
\begin{align}
x&=Rcos(\omega t)+Lsin(\phi)\\
y&=Rsin(\omega t)-Lcos(\phi)
\end{align}
således at:
\begin{align}
T&=\frac{m}{2}\big(L^2\dot{\phi}^2+R^2\omega^2+2RL\omega\dot{\phi}sin(\phi-\omega t)\big)\\
U&=mgL(1-cos(\phi))+mgRsin(\omega t)\\
\mathcal{L}(\phi,\dot{\phi},t)&=\frac{m}{2}\Big(L^2\dot{\phi}^2+R^2\omega^2+2RL\omega\dot{\phi}sin(\phi-\omega t)\Big)-mgL(1-cos(\phi))-mgRsin(\omega t)\\
\end{align}
bevægelseslign.:
\begin{align}
\frac{\partial \mathcal{L}}{\partial \phi}&=\frac{d}{dt}\bigg(\frac{\partial \mathcal{L}}{\partial \dot{\phi}}\bigg)\\
mRL\omega\dot{\phi}cos(\phi-\omega t)-sin(\phi)mgL&=mL^2\ddot{\phi}+mRL\omega cos(\phi-\omega t)(\dot{\phi}-\omega)\\
\ddot{\phi}&=\frac{R}{L}\omega^2cos(\phi-\omega t )-\frac{g}{L}sin(\phi)\\
\end{align}

In [ ]:
m=2
l = 2
R = 2
g = 9.82
omega = 2

tinit = 0
tfinal = 10
trange = [tinit,tfinal]

##startbetingelser
phi_0 = np.pi/3
phi_prik_0 =0

yinit = [phi_0,phi_prik_0]
ts = np.linspace(tinit, tfinal, 1000)

def dydt(t,y):
    phi = y[0]
    phi_prik = y[1]
    
    c1, c2 = np.cos(omega*t-phi), np.sin(phi)
    
    d_phi_dt = phi_prik
    d_phi_prik_dt = (R/l)*omega**2*c1-(g/l)*c2
    
    return [d_phi_dt,d_phi_prik_dt]

mysol = solve_ivp(dydt, trange, yinit, t_eval = ts,rtol=3e-14)
ts = mysol.t
phi = mysol.y[0]
phi_prik = mysol.y[1]



In [ ]:

plt.rc('font', size=15)
fig,ax = plt.subplots(1,2,figsize=(20,6))
ax[1].plot(phi,phi_prik)
ax[0].plot(ts,phi)
ax[0].grid(),ax[1].grid()

ax[1].set_xlabel('$\phi$')
ax[1].set_ylabel(r'$\frac{d\phi}{dt}=\dot{\phi}$')

ax[0].set_ylabel('$\phi$')
ax[0].set_xlabel('$t$')


## Tjekker energibevarelse

In [ ]:
#def energy(angle_vals,omega_val,time_vals,d_angle_dt_vals,mass,length,grav_acc,radius):
#    c1, c2, c3 = np.sin(angle_vals-omega_val*time_vals), np.cos(angle_vals), np.sin(omega_val*time_vals)
#    c4, c5, c6 = omega_val, d_angle_dt_vals, mass
#    c7, c8, c9 = length, grav_acc, radius
#    T = 
    
#    #T = (c6/2)*(c9**2*c4**2+c7**2*c5**2+2*c9*c4*c7*c5*c1)
#    #U = c6*c8*c9*(1-c2)-c6*c8*c9*c3
#    E_tot = U+T
#    return [T,U,E_tot]

#T = energy(phi,omega,ts,phi_prik,m,l,g,R)[0]
#U = energy(phi,omega,ts,phi_prik,m,l,g,R)[1]
#E_tot = energy(phi,omega,ts,phi_prik,m,l,g,R)[2]

#fig, ax = plt.subplots(figsize=(19,4))
#ax.plot(ts,T,color='red',label='Kinetisk energi',linestyle='--')
#ax.plot(ts,U,color='black',label='potentiel energi',linestyle='--')
#ax.plot(ts,E_tot,label=r'$E_{tot}=T+U$',linestyle='--')
#ax.hlines(0,ts[0],ts[-1]),ax.grid(), ax.legend(), ax.set_xlim(ts[0],ts[-1])



In [ ]:

def angle_to_cartesian1(omega_val,radius,time_vals):
    x = np.cos(omega_val*time_vals)*radius
    y = np.sin(omega_val*time_vals)*radius
    return [x,y]

def angle_to_cartesian2(angle_vals,omega_val,radius,length,time_vals):
    x = radius*np.cos(omega_val*time_vals)+length*np.sin(angle_vals)
    y = radius*np.sin(omega*time_vals)-length*np.cos(angle_vals)
    return [x,y]

xs1 = angle_to_cartesian1(omega,R,ts)[0].tolist()
ys1 = angle_to_cartesian1(omega,R,ts)[1].tolist()

xs2 = angle_to_cartesian2(phi,omega,R,l,ts)[0].tolist()
ys2 = angle_to_cartesian2(phi,omega,R,l,ts)[1].tolist()

fig, ax = plt.subplots(figsize=(8, 8))

dot1, = ax.plot([],[],'ro',ms=8)
line1, = ax.plot([],[],color='black')
dot2, = ax.plot([],[],'ro',ms=8)
line2, = ax.plot([],[],color='black')
tracer, = ax.plot([],[],linewidth = 1, color='green')

def update(i):
    dot1.set_data(xs1[i],ys1[i])
    line1.set_data([0,xs1[i]],[0,ys1[i]])
    dot2.set_data(xs2[i],ys2[i])
    line2.set_data([xs1[i],xs2[i]],[ys1[i],ys2[i]])
    tracer.set_data(xs2[0:i+1],ys2[0:i+1])
    
    return line1, dot1, line2, dot2, tracer

def draw_circle(x,radius):
    upper_part = np.sqrt(radius**2-x**2) 
    lower_part = -np.sqrt(radius**2-x**2) 
    return upper_part,lower_part

xs = np.linspace(-R,R,1000)
ax.plot(0,0,'bo', ms = 4)
ax.plot(xs,draw_circle(xs,R)[0],linestyle='--',color='black')
ax.plot(xs,draw_circle(xs,R)[1],linestyle='--',color='black')

ax.set_xlim(-7,7)
ax.set_ylim(-7,7)
anim = animation.FuncAnimation(fig,
                               update,
                               frames=len(ts),
                               interval=10,
                               blit=True,
                               repeat_delay=0)
anim